In [9]:
# === 1. KHAI BÁO THƯ VIỆN ===
import pandas as pd
import glob
import numpy as np
import os
import warnings

# Tắt các cảnh báo không quan trọng để output gọn gàng hơn
warnings.filterwarnings('ignore', category=FutureWarning)

# === 2. HÀM HỖ TRỢ (HELPER FUNCTION) ===
def convert_volume_to_numeric(volume_str):
    """
    Chuyển đổi chuỗi khối lượng giao dịch (ví dụ: '1.25M', '350.5K') sang dạng số.
    """
    volume_str = str(volume_str).strip().upper()
    if 'M' in volume_str:
        return float(volume_str.replace('M', '')) * 1_000_000
    elif 'K' in volume_str:
        return float(volume_str.replace('K', '')) * 1_000
    try:
        # Xử lý trường hợp số không có hậu tố
        return float(volume_str.replace(',', ''))
    except (ValueError, TypeError):
        return np.nan

# === 3. SCRIPT CHÍNH THỰC HIỆN PIPELINE ===
# Khởi tạo
current_directory = os.getcwd()
data_folder_path = os.path.join(current_directory, '00. data_goc')
file_pattern = os.path.join(data_folder_path, '*.csv')
csv_files = glob.glob(file_pattern)
list_of_dataframes = []

print(f"Bắt đầu quá trình tiền xử lý từ thư mục: {data_folder_path}\n")

# Vòng lặp qua từng tệp CSV
for file_path in csv_files:
    # Bỏ qua các tệp tạm hoặc file notebook
    if not os.path.basename(file_path).startswith('~') and file_path.endswith('.csv'):
        ticker = os.path.basename(file_path).split('.')[0]
        print(f"Đang xử lý: {ticker} Tệp: {os.path.basename(file_path)})")
        
        # Nạp dữ liệu
        df = pd.read_csv(file_path)
        
        # Thêm cột 'Ticker' để định danh
        df['Ticker'] = ticker
        
        # Chuẩn hóa định dạng các cột
        df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce')
        
        numeric_cols = ['Price', 'Open', 'High', 'Low']
        for col in numeric_cols:
            df[col] = df[col].astype(str).str.replace(',', '', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
        df['Vol.'] = df['Vol.'].apply(convert_volume_to_numeric)
        
        df['Change %'] = df['Change %'].astype(str).str.replace('%', '', regex=False)
        df['Change %'] = pd.to_numeric(df['Change %'], errors='coerce') / 100
        
        list_of_dataframes.append(df)

# Gộp, sắp xếp và làm sạch lần cuối
if list_of_dataframes:
    # Tạo dataset cuối cùng
    final_df = pd.concat(list_of_dataframes, ignore_index=True)
    final_df = final_df.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

    # Xử lý missing values
    final_df = final_df.groupby('Ticker').apply(lambda group: group.ffill().bfill())
    
    # Xử lý duplicates
    final_df.drop_duplicates(inplace=True)
    
    # Đặt lại index
    final_df = final_df.reset_index(drop=True)

    # === 4. KẾT QUẢ ===
    print("\nTIỀN XỬ LÝ DỮ LIỆU HOÀN TẤT!")
    print("Dataset cuối cùng đã được tạo thành công.")
    print("\nThông tin DataFrame cuối cùng:")
    final_df.info()
    
    print("\n5 dòng đầu tiên của dữ liệu đã làm sạch:")
    display(final_df.head())

    output_filename = "merged_stock_data.csv"
    print(f"\nĐang lưu dataset cuối cùng thành tệp: {output_filename}")
    
    # Lưu ra file CSV
    # index=False là quan trọng để không lưu cột chỉ số (0, 1, 2, 3...)
    final_df.to_csv(output_filename, index=False) 
    
    print("Lưu tệp thành công!")
else:
    print("Lỗi: Không tìm thấy tệp CSV nào để xử lý.")

Bắt đầu quá trình tiền xử lý từ thư mục: c:\Users\ZenBook\OneDrive\Desktop\PT S&P\Nhom5thanhvien\00. data_goc

Đang xử lý: FPT Tệp: FPT.csv)
Đang xử lý: HPG Tệp: HPG.csv)
Đang xử lý: KDH Tệp: KDH.csv)
Đang xử lý: PNJ Tệp: PNJ.csv)
Đang xử lý: VCB Tệp: VCB.csv)

TIỀN XỬ LÝ DỮ LIỆU HOÀN TẤT!
Dataset cuối cùng đã được tạo thành công.

Thông tin DataFrame cuối cùng:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12165 entries, 0 to 12164
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      12165 non-null  datetime64[ns]
 1   Price     12165 non-null  float64       
 2   Open      12165 non-null  float64       
 3   High      12165 non-null  float64       
 4   Low       12165 non-null  float64       
 5   Vol.      12165 non-null  float64       
 6   Change %  12165 non-null  float64       
 7   Ticker    12165 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 760.

C:\Users\ZenBook\AppData\Local\Temp\ipykernel_13268\345739279.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = final_df.groupby('Ticker').apply(lambda group: group.ffill().bfill())


,Date,Price,Open,High,Low,Vol.,Change %,Ticker
0,2016-01-04,8910.9,9022.8,9041.4,8910.9,2430000.0,-0.0104,FPT
1,2016-01-05,8910.9,8910.9,8948.2,8892.3,2960000.0,0.0000,FPT
2,2016-01-06,8929.6,8910.9,8966.8,8910.9,1800000.0,0.0021,FPT
3,2016-01-07,8761.8,8929.6,8929.6,8761.8,7440000.0,-0.0188,FPT
4,2016-01-08,8631.3,8724.5,8743.1,8575.4,7160000.0,-0.0149,FPT



Đang lưu dataset cuối cùng thành tệp: merged_stock_data.csv
Lưu tệp thành công!


In [ ]:
# Lưu DataFrame đã làm sạch ra tệp CSV
output_file_name = 'cleaned_stock_data.csv'
final_df.to_csv(output_file_name, index=False)

print(f"Đã lưu thành công dataset cuối cùng vào tệp: '{output_file_name}'")

Đã lưu thành công dataset cuối cùng vào tệp: 'cleaned_stock.csv'


In [4]:
# Sử dụng 3 dấu ngoặc kép """ để tạo một chuỗi đa dòng
pipeline_text = """
Bắt đầu
   |
   V
[5 Tệp CSV (FPT, VCB, HPG, KDH, PNJ)]
   |
   V
[Vòng lặp qua từng tệp]
   |
   |--> [1. Nạp dữ liệu vào DataFrame]
   |
   |--> [2. Thêm cột 'Ticker' để định danh]
   |
   |--> [3. Chuẩn hóa định dạng]
   |       |
   |       |--> Chuyển 'Date' -> datetime
   |       |--> Chuyển Cột Giá -> float (xóa dấu phẩy)
   |       |--> Chuyển 'Vol.' -> float (xử lý M, K)
   |       |--> Chuyển 'Change %' -> float (xóa %, chia 100)
   |
   |--> [4. Gộp 5 DataFrame đã xử lý]
   |
   V
[Dataset Thống nhất]
   |
   |--> [5. Sắp xếp dữ liệu theo Ticker và Ngày]
   |
   |--> [6. Xử lý Missing Values (Forward/Backward Fill)]
   |
   |--> [7. Xử lý Duplicates (Xóa dòng trùng)]
   |
   V
[Dataset Cuối Cùng (Sạch và Thống nhất)]
   |
   V
Kết thúc
"""

print(pipeline_text)


Bắt đầu
   |
   V
[5 Tệp CSV (FPT, VCB, HPG, KDH, PNJ)]
   |
   V
[Vòng lặp qua từng tệp]
   |
   |--> [1. Nạp dữ liệu vào DataFrame]
   |
   |--> [2. Thêm cột 'Ticker' để định danh]
   |
   |--> [3. Chuẩn hóa định dạng]
   |       |
   |       |--> Chuyển 'Date' -> datetime
   |       |--> Chuyển Cột Giá -> float (xóa dấu phẩy)
   |       |--> Chuyển 'Vol.' -> float (xử lý M, K)
   |       |--> Chuyển 'Change %' -> float (xóa %, chia 100)
   |
   |--> [4. Gộp 5 DataFrame đã xử lý]
   |
   V
[Dataset Thống nhất]
   |
   |--> [5. Sắp xếp dữ liệu theo Ticker và Ngày]
   |
   |--> [6. Xử lý Missing Values (Forward/Backward Fill)]
   |
   |--> [7. Xử lý Duplicates (Xóa dòng trùng)]
   |
   V
[Dataset Cuối Cùng (Sạch và Thống nhất)]
   |
   V
Kết thúc

